In [253]:
import pandas as pd
import numpy as np

In [254]:
df = pd.read_parquet("../data/matches.parquet")

In [255]:
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   round             2020 non-null   object        
 1   match_date        2016 non-null   datetime64[ns]
 2   home_team         2020 non-null   object        
 3   guest_team        2020 non-null   object        
 4   stadium           2020 non-null   object        
 5   date_added        2020 non-null   datetime64[ns]
 6   score_home_team   2016 non-null   Int64         
 7   score_guest_team  2016 non-null   Int64         
 8   winning_team      2016 non-null   object        
dtypes: Int64(2), datetime64[ns](2), object(5)
memory usage: 146.1+ KB
      round          match_date  home_team     guest_team         stadium  \
0  Rodada 1 2020-08-08 19:00:00  Fortaleza   Athletico-PR   Castelão (CE)   
1  Rodada 1 2020-08-08 19:30:00   Coritiba  Internacional   Couto Pereira  

In [256]:
df = df.dropna()

Historical performance: Rolling average of points per game for each team over last N matches (e.g., last 5-10 games)

In [257]:
import pandas as pd

# 1. Create a long-format performance DataFrame upfront
def create_performance_df(match_df):
    # Home team performances
    home = match_df[['match_date', 'home_team', 'score_home_team', 'score_guest_team', 'winning_team']].copy()
    home['team'] = home['home_team']
    home['opponent'] = home['score_guest_team']
    home['is_home'] = True
    home['points'] = home['winning_team'].map({'home': 3, 'draw': 1, 'guest': 0})
    
    # Away team performances
    away = match_df[['match_date', 'guest_team', 'score_home_team', 'score_guest_team', 'winning_team']].copy()
    away['team'] = away['guest_team']
    away['opponent'] = away['score_home_team']
    away['is_home'] = False
    away['points'] = away['winning_team'].map({'home': 0, 'draw': 1, 'guest': 3})
    
    # Combine and sort
    perf_df = pd.concat([home, away], ignore_index=True)
    perf_df = perf_df.sort_values(['team', 'match_date'])
    
    # Calculate match sequence number for each team
    perf_df['team_match_seq'] = perf_df.groupby('team').cumcount() + 1
    
    return perf_df[['match_date', 'team', 'opponent', 'is_home', 'points', 'team_match_seq']]

# 2. Create feature calculation functions
def calculate_features(perf_df, window=5):
    # Ensure chronological order
    perf_df = perf_df.sort_values(['team', 'match_date'])
    
    # Rolling features
    perf_df['rolling_points_avg'] = perf_df.groupby('team')['points'].transform(
        lambda x: x.rolling(window, min_periods=1).mean().shift(1))
    
    perf_df['rolling_points_std'] = perf_df.groupby('team')['points'].transform(
        lambda x: x.rolling(window, min_periods=1).std().shift(1))
    
    # Can add more features here (form, streaks, etc.)
    return perf_df

# 3. Create match-level features by joining performance data
def create_match_features(match_df, perf_df):
    # Home team features
    match_df = match_df.merge(
        perf_df.add_prefix('home_'),
        left_on=['match_date', 'home_team'],
        right_on=['home_match_date', 'home_team'],
        how='left'
    )
    
    # Away team features
    match_df = match_df.merge(
        perf_df.add_prefix('away_'),
        left_on=['match_date', 'guest_team'],
        right_on=['away_match_date', 'away_team'],
        how='left'
    )
    
    return match_df

# Main workflow
# Assuming df is your original DataFrame
perf_df = create_performance_df(df)
featured_perf_df = calculate_features(perf_df)
final_df = create_match_features(df, featured_perf_df)

Last 5 games win/draw/loss ratio for both teams

In [258]:
import pandas as pd
from collections import defaultdict

# Assuming final_df is your dataframe
# First sort by match date to ensure chronological order
final_df = final_df.sort_values('match_date')

# Create dictionaries to store team histories
team_history = defaultdict(list)

# Function to calculate last 5 matches performance
def get_last_5_performance(team, date):
    team_matches = team_history[team]
    last_5 = [m for m in team_matches if m['date'] < date][-5:]
    
    if not last_5:
        return {'win_ratio': None, 'draw_ratio': None, 'loss_ratio': None}
    
    wins = 0
    draws = 0
    losses = 0
    
    for match in last_5:
        if match['result'] == 'win':
            wins += 1
        elif match['result'] == 'draw':
            draws += 1
        else:
            losses += 1
    
    total = wins + draws + losses
    return {
        'win_ratio': wins / total,
        'draw_ratio': draws / total,
        'loss_ratio': losses / total
    }

# First pass to populate team histories
for _, row in final_df.iterrows():
    if pd.isna(row['match_date']) or pd.isna(row['winning_team']):
        continue
    
    home_team = row['home_team']
    guest_team = row['guest_team']
    match_date = row['match_date']
    
    # Determine results for each team
    if row['winning_team'] == 'home':
        home_result = 'win'
        guest_result = 'loss'
    elif row['winning_team'] == 'guest':
        home_result = 'loss'
        guest_result = 'win'
    else:  # draw
        home_result = 'draw'
        guest_result = 'draw'
    
    # Add to team histories
    team_history[home_team].append({
        'date': match_date,
        'result': home_result,
        'opponent': guest_team,
        'home_away': 'home'
    })
    
    team_history[guest_team].append({
        'date': match_date,
        'result': guest_result,
        'opponent': home_team,
        'home_away': 'away'
    })

# Second pass to calculate last 5 performance
home_win_ratios = []
home_draw_ratios = []
home_loss_ratios = []
guest_win_ratios = []
guest_draw_ratios = []
guest_loss_ratios = []

for _, row in final_df.iterrows():
    if pd.isna(row['match_date']):
        home_win_ratios.append(None)
        home_draw_ratios.append(None)
        home_loss_ratios.append(None)
        guest_win_ratios.append(None)
        guest_draw_ratios.append(None)
        guest_loss_ratios.append(None)
        continue
    
    home_perf = get_last_5_performance(row['home_team'], row['match_date'])
    guest_perf = get_last_5_performance(row['guest_team'], row['match_date'])
    
    home_win_ratios.append(home_perf['win_ratio'])
    home_draw_ratios.append(home_perf['draw_ratio'])
    home_loss_ratios.append(home_perf['loss_ratio'])
    
    guest_win_ratios.append(guest_perf['win_ratio'])
    guest_draw_ratios.append(guest_perf['draw_ratio'])
    guest_loss_ratios.append(guest_perf['loss_ratio'])

# Add new columns to dataframe
final_df['home_win_ratio_last5'] = home_win_ratios
final_df['home_draw_ratio_last5'] = home_draw_ratios
final_df['home_loss_ratio_last5'] = home_loss_ratios
final_df['guest_win_ratio_last5'] = guest_win_ratios
final_df['guest_draw_ratio_last5'] = guest_draw_ratios
final_df['guest_loss_ratio_last5'] = guest_loss_ratios

Goals scored/conceded in last N matches

In [259]:
# Assuming your dataframe is named 'df'
# First, sort by match_date to ensure chronological order
final_df = final_df.sort_values('match_date').reset_index(drop=True)

# Define the number of previous matches to consider
N = 5  # You can adjust this based on your needs

# Create a long format dataframe with all team performances
home_games = final_df[['match_date', 'home_team', 'score_home_team', 'score_guest_team']].copy()
home_games['is_home'] = True
home_games = home_games.rename(columns={
    'home_team': 'team',
    'score_home_team': 'goals_scored',
    'score_guest_team': 'goals_conceded'
})

away_games = final_df[['match_date', 'guest_team', 'score_guest_team', 'score_home_team']].copy()
away_games['is_home'] = False
away_games = away_games.rename(columns={
    'guest_team': 'team',
    'score_guest_team': 'goals_scored',
    'score_home_team': 'goals_conceded'
})

all_games = pd.concat([home_games, away_games]).sort_values(['team', 'match_date'])

# Calculate rolling averages
for team in all_games['team'].unique():
    team_mask = all_games['team'] == team
    all_games.loc[team_mask, 'goals_scored_avg'] = (
        all_games.loc[team_mask, 'goals_scored']
        .rolling(N, min_periods=1)
        .mean()
        .shift(1)  # Use previous matches only
    )
    all_games.loc[team_mask, 'goals_conceded_avg'] = (
        all_games.loc[team_mask, 'goals_conceded']
        .rolling(N, min_periods=1)
        .mean()
        .shift(1)
    )

# Merge back to original dataframe
# For home teams
final_df = final_df.merge(
    all_games[all_games['is_home']][['match_date', 'team', 'goals_scored_avg', 'goals_conceded_avg']],
    left_on=['match_date', 'home_team'],
    right_on=['match_date', 'team'],
    how='left'
).rename(columns={
    'goals_scored_avg': f'home_team_goals_scored_last_{N}',
    'goals_conceded_avg': f'home_team_goals_conceded_last_{N}'
}).drop(columns=['team'])

# For away teams
final_df = final_df.merge(
    all_games[~all_games['is_home']][['match_date', 'team', 'goals_scored_avg', 'goals_conceded_avg']],
    left_on=['match_date', 'guest_team'],
    right_on=['match_date', 'team'],
    how='left'
).rename(columns={
    'goals_scored_avg': f'guest_team_goals_scored_last_{N}',
    'goals_conceded_avg': f'guest_team_goals_conceded_last_{N}'
}).drop(columns=['team'])

Clean sheet frequency (games without conceding)

In [260]:
def calculate_cs_frequency(df, window=5):
    """
    Calculate clean sheet frequency for last N matches regardless of home/away status
    Returns two Series: (home_team_cs, guest_team_cs)
    """
    # Initialize dictionaries to track team histories
    team_history = {}
    home_cs = pd.Series(np.nan, index=df.index)
    guest_cs = pd.Series(np.nan, index=df.index)
    
    # First collect all matches for each team
    all_matches = []
    
    for idx, row in df.iterrows():
        # For home team perspective
        all_matches.append({
            'team': row['home_team'],
            'date': row['match_date'],
            'goals_conceded': row['score_guest_team'],
            'idx': idx,
            'type': 'home'
        })
        # For away team perspective
        all_matches.append({
            'team': row['guest_team'],
            'date': row['match_date'],
            'goals_conceded': row['score_home_team'],
            'idx': idx,
            'type': 'guest'
        })
    
    # Sort all matches by date
    all_matches_sorted = sorted(all_matches, key=lambda x: x['date'])
    
    # Process each team separately
    for team in df['home_team'].unique():
        team_matches = [m for m in all_matches_sorted if m['team'] == team]
        
        for i, match in enumerate(team_matches):
            prev_matches = team_matches[max(0, i-window):i]
            
            if prev_matches:
                clean_sheets = sum(1 for m in prev_matches if m['goals_conceded'] == 0)
                freq = clean_sheets / len(prev_matches)
            else:
                freq = 0
            
            # Assign to appropriate series
            if match['type'] == 'home':
                home_cs[match['idx']] = freq
            else:
                guest_cs[match['idx']] = freq
    
    return home_cs, guest_cs

# Calculate clean sheet frequencies
final_df['home_team_cs_freq_last5'], final_df['guest_team_cs_freq_last5'] = calculate_cs_frequency(final_df, window=5)

Historical H2H record between the two teams (win %)

In [261]:
def calculate_correct_h2h_win_percentage(df, window=5):
    """
    Correctly calculates head-to-head win percentages for both teams
    Returns: home_team_h2h_win_pct, guest_team_h2h_win_pct, h2h_draw_pct
    """
    # Initialize results
    home_win_pct = pd.Series(0.0, index=df.index)
    guest_win_pct = pd.Series(0.0, index=df.index)
    draw_pct = pd.Series(0.0, index=df.index)
    
    # Dictionary to store all matches between team pairs
    h2h_history = {}
    
    # Sort dataframe by match date to process in chronological order
    df_sorted = df.sort_values('match_date').reset_index(drop=True)
    
    for idx, row in df_sorted.iterrows():
        home = row['home_team']
        guest = row['guest_team']
        
        # Create consistent team pair key (sorted alphabetically)
        team_pair = tuple(sorted((home, guest)))
        
        if team_pair not in h2h_history:
            h2h_history[team_pair] = []
        
        # Get previous matches between these teams (excluding current match)
        prev_matches = h2h_history[team_pair][-window:]
        
        # Initialize counters
        home_wins = 0
        guest_wins = 0
        draws = 0
        
        # Analyze previous matches
        for match in prev_matches:
            if match['winner'] == 'home':
                if match['home_team'] == home:
                    home_wins += 1
                else:
                    guest_wins += 1
            elif match['winner'] == 'guest':
                if match['guest_team'] == home:
                    home_wins += 1
                else:
                    guest_wins += 1
            else:
                draws += 1
        
        # Calculate percentages
        total_matches = len(prev_matches)
        if total_matches > 0:
            home_win_pct[idx] = home_wins / total_matches
            guest_win_pct[idx] = guest_wins / total_matches
            draw_pct[idx] = draws / total_matches
        
        # Store current match in history
        h2h_history[team_pair].append({
            'home_team': home,
            'guest_team': guest,
            'winner': row['winning_team']
        })
    
    # Reindex to match original dataframe order
    home_win_pct = home_win_pct.reindex(df.index)
    guest_win_pct = guest_win_pct.reindex(df.index)
    draw_pct = draw_pct.reindex(df.index)
    
    return home_win_pct, guest_win_pct, draw_pct

# Calculate the correct H2H percentages
final_df['home_team_h2h_win_pct'], final_df['guest_team_h2h_win_pct'], final_df['h2h_draw_pct'] = \
    calculate_correct_h2h_win_percentage(final_df, window=5)

Goal difference trend in H2H matches

In [262]:
def calculate_h2h_goal_diff_trend(df, window=5):
    """
    Calculate goal difference trend in last N H2H matches between teams
    Returns two Series: (home_team_h2h_gd_trend, guest_team_h2h_gd_trend)
    """
    # Initialize results
    home_gd_trend = pd.Series(0.0, index=df.index)
    guest_gd_trend = pd.Series(0.0, index=df.index)
    
    # Dictionary to store all matches between team pairs
    h2h_history = {}
    
    # Sort dataframe by match date to process in chronological order
    df_sorted = df.sort_values('match_date').reset_index(drop=True)
    
    for idx, row in df_sorted.iterrows():
        home = row['home_team']
        guest = row['guest_team']
        
        # Create consistent team pair key (sorted alphabetically)
        team_pair = tuple(sorted((home, guest)))
        
        if team_pair not in h2h_history:
            h2h_history[team_pair] = []
        
        # Get previous matches between these teams (excluding current match)
        prev_matches = h2h_history[team_pair][-window:]
        
        # Initialize goal difference accumulators
        home_gd = 0
        guest_gd = 0
        
        # Analyze previous matches
        for match in prev_matches:
            if match['home_team'] == home:
                # Current home team was home in this historical match
                home_gd += match['home_score'] - match['guest_score']
                guest_gd += match['guest_score'] - match['home_score']
            else:
                # Current home team was away in this historical match
                home_gd += match['guest_score'] - match['home_score']
                guest_gd += match['home_score'] - match['guest_score']
        
        # Calculate average goal difference
        if prev_matches:
            home_gd_trend[idx] = home_gd / len(prev_matches)
            guest_gd_trend[idx] = guest_gd / len(prev_matches)
        
        # Store current match in history
        h2h_history[team_pair].append({
            'home_team': home,
            'guest_team': guest,
            'home_score': row['score_home_team'],
            'guest_score': row['score_guest_team']
        })
    
    # Reindex to match original dataframe order
    home_gd_trend = home_gd_trend.reindex(df.index)
    guest_gd_trend = guest_gd_trend.reindex(df.index)
    
    return home_gd_trend, guest_gd_trend

# Calculate H2H goal difference trends
final_df['home_team_h2h_gd_trend'], final_df['guest_team_h2h_gd_trend'] = \
    calculate_h2h_goal_diff_trend(final_df, window=5)

In [263]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   round                             2016 non-null   object        
 1   match_date                        2016 non-null   datetime64[ns]
 2   home_team                         2016 non-null   object        
 3   guest_team                        2016 non-null   object        
 4   stadium                           2016 non-null   object        
 5   date_added                        2016 non-null   datetime64[ns]
 6   score_home_team                   2016 non-null   Int64         
 7   score_guest_team                  2016 non-null   Int64         
 8   winning_team                      2016 non-null   object        
 9   home_match_date                   2016 non-null   datetime64[ns]
 10  home_opponent                     2016 non-null 

In [264]:
mask =  final_df['home_team'].isin(['Flamengo', 'Fluminense']) & final_df['guest_team'].isin(['Flamengo', "Fluminense"])

final_df[mask][['home_team','guest_team', 'winning_team', 'score_home_team', 'score_guest_team', 'home_team_h2h_gd_trend', 'guest_team_h2h_gd_trend']].head(10)

,home_team,guest_team,winning_team,score_home_team,score_guest_team,home_team_h2h_gd_trend,guest_team_h2h_gd_trend
79,Fluminense,Flamengo,guest,1,2,0.000000,0.000000
273,Flamengo,Fluminense,guest,1,2,1.000000,-1.000000
461,Flamengo,Fluminense,guest,0,1,0.000000,0.000000
646,Fluminense,Flamengo,home,3,1,0.333333,-0.333333
834,Fluminense,Flamengo,guest,1,2,0.750000,-0.750000
1023,Flamengo,Fluminense,guest,1,2,-0.400000,0.400000
1282,Fluminense,Flamengo,draw,0,0,0.800000,-0.800000
1466,Flamengo,Fluminense,draw,1,1,-0.600000,0.600000
1619,Fluminense,Flamengo,guest,0,1,0.400000,-0.400000
1808,Flamengo,Fluminense,guest,0,2,0.200000,-0.200000


In [265]:
# filter flamengo in home or guest team
final_df[(final_df['home_team'] == 'Flamengo') | (final_df['guest_team'] == 'Flamengo')][['home_team','guest_team', 'score_home_team', 'score_guest_team', 'home_team_cs_freq_last5', 
          'guest_team_cs_freq_last5']].head(10)

,home_team,guest_team,score_home_team,score_guest_team,home_team_cs_freq_last5,guest_team_cs_freq_last5
3,Flamengo,Atlético-MG,0,1,0.000000,0.000000
10,Atlético-GO,Flamengo,3,0,0.000000,0.000000
17,Coritiba,Flamengo,0,1,0.000000,0.000000
26,Flamengo,Grêmio,1,1,0.333333,0.666667
39,Flamengo,Botafogo,1,1,0.250000,0.333333
53,Santos,Flamengo,0,1,0.200000,0.200000
59,Bahia,Flamengo,3,5,0.200000,0.400000
65,Flamengo,Fortaleza,2,1,0.400000,0.400000
79,Fluminense,Flamengo,1,2,0.200000,0.200000
91,Ceará,Flamengo,2,0,0.600000,0.200000


In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load data (assuming `final_df` from previous steps)
df = final_df.copy()

# Encode target variable (winning_team: home/draw/guest)
le = LabelEncoder()
df['target'] = le.fit_transform(df['winning_team'])  # 0=home, 1=draw, 2=guest

# Select features (modify as needed)
features = [
    'home_rolling_points_avg', 'away_rolling_points_avg',

    'home_win_ratio_last5', 'home_draw_ratio_last5', 'home_loss_ratio_last5', 'guest_win_ratio_last5', 'guest_draw_ratio_last5', 'guest_loss_ratio_last5',

    'home_team_goals_scored_last_5', 'home_team_goals_conceded_last_5', 'guest_team_goals_scored_last_5', 'guest_team_goals_conceded_last_5',

    'home_team_cs_freq_last5', 'guest_team_cs_freq_last5',

    'home_team_h2h_win_pct', 'guest_team_h2h_win_pct', 'h2h_draw_pct',

    'home_team_h2h_gd_trend', 'guest_team_h2h_gd_trend'
]

X = df[features]
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Calculate class weights (inverse of class frequencies)
class_weights = len(y_train) / (3 * np.bincount(y_train))
sample_weights = np.array([class_weights[y] for y in y_train])

model = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    random_state=42
)

# Pass sample_weight during training
model.fit(X_train, y_train, sample_weight=sample_weights)

# Predictions
y_pred = model.predict(X_test)

# Decode labels back to original values
class_names = le.inverse_transform([0, 1, 2])  # ['home', 'draw', 'guest']

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

print(classification_report(
    y_test,
    y_pred,
    target_names=class_names,
    digits=3
))

# Get feature importances
importance = model.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': importance
}).sort_values('Importance', ascending=False)

print(feature_importance)

[[26 36 41]
 [22 46 37]
 [47 66 83]]
              precision    recall  f1-score   support

        draw      0.274     0.252     0.263       103
       guest      0.311     0.438     0.364       105
        home      0.516     0.423     0.465       196

    accuracy                          0.384       404
   macro avg      0.367     0.371     0.364       404
weighted avg      0.401     0.384     0.387       404

                             Feature  Importance
2               home_win_ratio_last5    0.085595
14             home_team_h2h_win_pct    0.069756
15            guest_team_h2h_win_pct    0.060590
4              home_loss_ratio_last5    0.058777
1            away_rolling_points_avg    0.058630
13          guest_team_cs_freq_last5    0.057037
5              guest_win_ratio_last5    0.056753
8      home_team_goals_scored_last_5    0.056060
0            home_rolling_points_avg    0.055881
11  guest_team_goals_conceded_last_5    0.053950
3              home_draw_ratio_last5    0.0